In [7]:
#This kernel defines all the functions that we will need for the Grobner basis algorithm
#Here, we define the free tensor algebra in which we are working in. a, b, c, d... represent l1, r1, l2, r2... respectively.
#We order this lexographically, which is the same as the ordering of the li and ri in the paper.

######################################################################
#BELOW SPECIFY RING YOU ARE WORKING WITH AND MAX NUMBBER OF VARIABLES#
######################################################################

A.<r,q,p,o,n,m,l,k,j,i,h,g,f,e,d,c,b,a> = FreeAlgebra(GF(2), 18, order = "lex")
#lv is the list that includes the variables that represent the l1, l2...
#rv is the list that includes the variables that represent the r1, r2...
lv = [a,c,e,g,i,k,m,o,q]
rv = [b,d,f,h,j,l,n,p,r]
#this dictionary converts the string forms of the variables into the actual variables in the free tensor algebra
#the str() function converts the actual variables into their string forms. Working with strings is much faster.
invdict = {"":1, "a":a, "b":b, "c":c, "d":d, "e":e, "f":f, "g":g, "h":h, "i":i, "j":j, "k":k, "l":l, "m":m, "n":n, "o":o, "p":p, "q":q, "r":r}

#For these functions, I am using f and g to denote polynomials; a and b to denote monomials. This is different from the variables a, b, f, g
#input: polynomial in the free tensor algebra
#output: leading monomial
def lm(f):
    if(f==0):
        return 0
    ans = f.leading_monomial()
    for x in f.monomials():
        if (x>ans):
            ans = x
    return ans

#input: string representing a monomial
#output: translation into a monomial in the free tensor algebra
#e.g.: "abc" -> a*b*c
def invstring(s):
    ans = 1
    for i in range(len(s)):
        ans = ans*invdict[s[i:i+1]]
    return ans

#input: monomial in the free tensor algebra
#output: string representation of the monomial
#note that this is tricky because the str() function will preserve * and ^ symbols
#e.g.: a*b^2*c -> "abbc"
def forstring(a):
    count = -1
    ans = ""
    try: 
        s = str(a)
    except: 
        print(a)
    s = s+"*"
    for i in range(len(s)):
        if s[i:i+1]=="*":
            if i == count + 2:
                ans = ans + s[i-1:i]
            if i == count + 4:
                for j in range(int(s[i-1:i])):
                    ans = ans + s[i-3:i-2]
            count = i
    return ans

#input: polynomial in the free tensor algebra
#output: leading coefficient
def lc(f):
    if(f==0):
        return 0
    #the coefficients are ordered opposite to how we would write the polynomial somehow, so the leading coefficient is the last one
    return f.coefficients()[-1]

#input: polynomial in the free tensor algebra
#output: degree of the polynomial
def deg(f):
    a=lm(f)
    return len(forstring(a))

#input: two polynomials in the free tensor algebra
#output: whether the lm(f) divides lm(g)
def divides(f,g):
    a = lm(f)
    if a==1:
        return true
    b = lm(g)
    astr = forstring(a)
    bstr = forstring(b)
    return bstr.find(astr) >= 0

#precondition: divides(f,g)
#input: two polynomials in the free tensor algebra
#output: two monomials x, y such that x*lm(f)*y = lm(g)
def divisor(f,g):
    a = lm(f)
    b = lm(g)
    astr = forstring(a)
    bstr = forstring(b)
    c = invstring(bstr[0:bstr.find(astr)])
    d = invstring(bstr[bstr.find(astr)+len(astr):len(bstr)])
    return [c,d]

#input: polynomial in the free tensor algebra
#output: list of left divisors of lm(f)
#e.g.: a*b*c -> [1, a, a*b, a*b*c]
def left_divisors(f):
    a = lm(f)
    ans = []
    astr = forstring(a)
    for i in range(len(astr)+1):
        ans.append(invstring(astr[:i]))
    return ans

#input: polynomial in the free tensor algebra
#output: list of right divisors of lm(f)
#e.g.: a*b*c -> [1, c, b*c, a*b*c]
def right_divisors(f):
    a = lm(f)
    ans = [1]
    astr = forstring(a)
    for i in range(1,len(astr)+1):
        ans.append(invstring(astr[-i:]))
    return ans

#input: two polynomials in the free tensor algebra
#output: monomial x such that x*lm(f) = lm(g)
def left_quotient(f,g):
    a = lm(f)
    b = lm(g)
    astr = forstring(a)
    bstr = forstring(b)
    return invstring(bstr[0:len(bstr)-len(astr)])

#input: two polynomials in the free tensor algebra
#output: monomial x such that lm(f)*x = lm(g)
def right_quotient(f,g):
    a = lm(f)
    b = lm(g)
    astr = forstring(a)
    bstr = forstring(b)
    return invstring(bstr[len(astr):len(bstr)])

#input: two polynomials in the free tensor algebra
#output: monomial x that is the greatest overlap between lm(f) and lm(g)
def gcd(f,g):
    a = lm(f)
    b = lm(g)
    ans = 1
    for x in right_divisors(a):
        if x in left_divisors(b):
            if x > ans:
                ans = x
    return ans

In [ ]:
#We need some function that handles the multiplication
#Of course, if the algebra is homogeneous (trivial multiplication), then any mult[0][(i,j)] or mult[1][(i,j)] is just 0

####################################################################
#DEINFE YOUR MULTIPLICATION HERE, EITHER EXPLICITLY, OR WITH A CODE#
####################################################################

#this is just initializing multiplication for the Octonions
mult = {0: {}, 1:{}}
#0 is for left multiplication: mult[0][(i,j)] = l_{i*j}
#1 is for right multiplication: mult[1][(i,j)] = r_{i*j}
varKey = {0:{0: a, 1: c, 2: e, 3: g, 4: i, 5: k, 6: m, 7: o}, 
          1:{0: b, 1: d, 2: f, 3: h, 4: j, 5: l, 6: n, 7: p}}
"""
0 is (1,0)
1 is (i,0)
2 is (j,0)
3 is (k,0)
4 is (0,1)
5 is (0,k)
6 is (0,j)
7 is (0,i)
"""
smallMult = {(1,2): 3, (2,3): 1, (3,1): 2}
def initMult(sign):
    for i in range(8):
        for j in range(8):
            if (i == 0 or j == 0):
                mult[sign].update({(i,j):varKey[sign][i+j]})
            elif (i == 4 and j == 4):
                mult[sign].update({(i,j):-varKey[sign][0]})
            elif (i == 4):
                if (j < 4):
                    mult[sign].update({(i,j): -varKey[sign][8-j]})
                else:
                    mult[sign].update({(i,j): varKey[sign][8-j]})
            elif (j == 4):
                if (i < 4):
                    mult[sign].update({(i,j): varKey[sign][8-i]})
                else:
                    mult[sign].update({(i,j): -varKey[sign][8-i]})
            elif (i < 4):
                if (j == i):
                    mult[sign].update({(i,j): -varKey[sign][0]})
                elif (j == 8-i):
                    mult[sign].update({(i,j): -varKey[sign][4]})
                elif (i,j) in smallMult.keys():
                    mult[sign].update({(i,j): varKey[sign][smallMult[(i,j)]]})
                elif (j,i) in smallMult.keys():
                    mult[sign].update({(i,j): -varKey[sign][smallMult[(j,i)]]})
                elif (8-j,i) in smallMult.keys():
                    mult[sign].update({(i,j): varKey[sign][8-smallMult[(8-j,i)]]})
                else:
                    mult[sign].update({(i,j): -varKey[sign][8-smallMult[(i,8-j)]]})
            else:
                if (j == i):
                    mult[sign].update({(i,j): -varKey[sign][0]})
                elif (j == 8-i):
                    mult[sign].update({(i,j): varKey[sign][4]})
                elif (8-i, j) in smallMult.keys():
                    mult[sign].update({(i,j): -varKey[sign][8-smallMult[(8-i,j)]]})
                elif (j, 8-i) in smallMult.keys():
                    mult[sign].update({(i,j): varKey[sign][8-smallMult[(j,8-i)]]})
                elif (8-j, 8-i) in smallMult.keys():
                    mult[sign].update({(i,j): varKey[sign][smallMult[(8-j,8-i)]]})
                else:
                    mult[sign].update({(i,j): -varKey[sign][smallMult[(8-i,8-j)]]})
initMult(0)
initMult(1)


In [ ]:
#This kernel initializes I
#rank of free K-module. We can alter this up to 9 because the free tensor algebra we defined includes l1, r1... l9, r9. But of course, you can increase max number of variables in the first kernel

##################################
#SPECIFY THE RANK OF YOUR ALGEBRA#
##################################

rank = 8

#constructs the initial basis for I, which is not enlarged/complete yet
I = []
def construct_I():
    #this constructs I
    for i in range(rank):
        I.append(lv[i]^2-mult[0][(i,i)])
        I.append(rv[i]^2-mult[1][(i,i)])
        I.append(-rv[i]*lv[i]+lv[i]*rv[i])
    for i in range(rank):
        for j in range(i+1,rank):
            I.append(mult[0][(i,j)]-lv[j]*lv[i]-rv[j]*lv[i]+lv[i]*rv[j])
            I.append(rv[i]*rv[j]-mult[1][(i,j)]-rv[j]*lv[i]+lv[i]*rv[j])
            I.append(mult[0][(j,i)]-lv[i]*lv[j]-rv[i]*lv[j]+lv[j]*rv[i])
            I.append(rv[j]*rv[i]-mult[1][(j,i)]-rv[i]*lv[j]+lv[j]*rv[i])
            I.append(lv[i]*lv[j]+lv[j]*lv[i]-mult[0][(i,j)]-mult[0][(j,i)])
            I.append(rv[i]*rv[j]+rv[j]*rv[i]-mult[1][(i,j)]-mult[1][(j,i)])
construct_I()
print(I)
print(len(I))

In [ ]:
#This cell defines full reduction

#reduces f with respect to Y\{f}
def reduceN(f, Y, N):
    if (f==0):
        return f
    temp = f
    flag = False
    for x in Y:
        if (x == N):
            continue
        if divides(lm(x),lm(temp)):
            temp = temp-lc(temp)/lc(x)*divisor(lm(x),lm(temp))[0]*x*divisor(lm(x),lm(temp))[1]
            flag = True
            break
    if (flag==False):
        return f
    return reduceN(temp, Y, N)

#reduces f and all the monomials of f (so not just the leading monomial) with respect to Y\{f}
def fullReduceN(f, Y, N):
    if (f == 0):
        return f
    temp = reduceN(f, Y, N)
    return lm(temp)*lc(temp) + fullReduceN(temp - lm(temp)*lc(temp), Y, N)

In [ ]:
#This simplifies I until it cannot be simplified anymore
def simplify():
    sorted = False
    while (sorted == False):
        x = 0
        while (x<len(I)):
            if (I[x] == 0):
                I.pop(x)
            else:
                x+=1
        sorted = True
        x = 0
        while (x<len(I)):
            if (x%20 == 0):
                print("CURRENTLY AT " + str(x))
            tempIx = fullReduceN(I[x],I,I[x])
            if (tempIx != I[x]):
                sorted = False
                I[x] = tempIx 
                print(tempIx)
            x+=1
simplify()
print(I)
print(len(I))

In [ ]:
#This cell removes 0s
x = 0
while (x<len(I)):
    if (I[x] == 0):
        I.pop(x)
    else:
        x+=1
print(len(I))

In [ ]:
#This cell does the Buchberger algorithm

#############################################################################################################
#THE CURRENT ALGORITHM IS TO GET GROBNER BASIS, THEN SIMPLIFY. THIS MAY BE VERY SLOW; ONE STRATEGY IS TO DO #
#THE BUCHBERGER ALGORITHM FOR THE FIRST SAY 200 PAIRS, SIMPLIFY, THEN DO THE BUCHBERGER ALGORITHM, SIMPLIFY,#
#AND SO ON... UNTIL YOU GET A GROBNER BASIS. SIMPLIFYING IN THE MIDDLE ENSURES THAT THE ELEMENTS OF I ARE   #
#NEVER TOO COMPLICATED                                                                                      #
#############################################################################################################

#reduces f with respect to Y
def reduce(f, Y):
    if (f==0):
        return f
    temp = f
    flag = False
    for x in Y:
        if divides(lm(x),lm(temp)):
            temp = temp-lc(temp)/lc(x)*divisor(lm(x),lm(temp))[0]*x*divisor(lm(x),lm(temp))[1]
            flag = True
            break
    if (flag==False):
        return f
    return reduce(temp, Y)

#reduces f and all the monomials of f with respect to Y
def fullReduce(f, Y):
    if (f==0):
        return f
    temp = reduce(f, Y)
    return lc(temp)*lm(temp) + fullReduce(temp - lc(temp)*lm(temp), Y)

#input: two polynomials in the free tensor algebra
#output: the S-polynomial between f and g with respect to basis I
def Spoly(f,g,Y):
    b = gcd(f,g)
    if (b==1):
        return 0
    a = left_quotient(b,f)
    c = right_quotient(b,g)
    return fullReduce(a*g/lc(g)-f*c/lc(f), Y)

#updates the ideal I via the Grobner basis algorithm
def grobner():
    i = 0
    while (i < len(I)):
        j=0
        print("Checking "+str(i)+", Total "+str(len(I)))
        while j<i:
            temp = Spoly(I[i],I[j], I)
            if (temp != 0):
                I.append(temp)
            temp = Spoly(I[j],I[i], I)
            if (temp != 0):
                I.append(temp)
            j+=1
        i+=1 

grobner()
print(I)
print(len(I))

#then simplify
simplify()
print(I)
print(len(I))

In [ ]:
#this cell puts the leading monomials into K
K = I[:]
def lmList():
    for x in range(len(K)):
        K[x] = lm(K[x])
lmList()
print(K)
print(len(K))

In [ ]:
#this cell orders the leading monomials and puts them into M
def orderK():
    theSum = 0
    for x in K:
        theSum += x
    return theSum
temp = orderK()
M = []
while(temp != 0):
    M.insert(0, lm(temp))
    temp -= lm(temp)
print(M)
print(len(M))